In [1]:
import sys, random, os, json

sys.path.append('/Users/richardtran/repos/Shell_repo/')
from structure_generation.bare_slabs import slab_generator
from structure_generation.lmdb_generator import generate_lmdb
from structure_generation.oxide_adsorption import surface_adsorption_saturation

sys.path.append('/Users/richardtran/repos/ocp')
from ocpmodels.datasets import LmdbDataset

bulk_json = '/Users/richardtran/repos/Shell_repo/database/bulk_oxides_20220621.json'
bulk_oxides_20220621 = json.load(open(bulk_json, 'rb'))
bulk_oxides_dict = {entry['entry_id']: entry for entry in bulk_oxides_20220621}

# Get lmdb for the bare slabs

In [2]:
slabs = slab_generator('mp-8840', 1, 15, 15)
generate_lmdb(slabs, 'test.lmdb')
bare_slab_dataset = LmdbDataset({'src': 'test.lmdb'})

# Get lmdb for the adsorbed slabs of one bare slab

In [3]:
dat = bare_slab_dataset[0]
adslabs = surface_adsorption_saturation(dat)
print(len(adslabs))
generate_lmdb(adslabs, 'test_ads.lmdb')
adslab_dataset = LmdbDataset({'src': 'test_ads.lmdb'})

Found total adsites: 1 for slab of 112 atoms in 0.102 secs
coverage 1
partition_combos (0,) Total combos 4
groups 4 combinations into 4 groups
grouped 1 sites to 1
symmetrically reduced to 1 adslabs from 1
removed 0.0 adsorbates! original coverage 1
sorting 1 adslabs, completed 0/4 iterations
grouped 1 sites to 1
symmetrically reduced to 1 adslabs from 1
removed 0.0 adsorbates! original coverage 1
sorting 2 adslabs, completed 1/4 iterations
grouped 1 sites to 1
symmetrically reduced to 1 adslabs from 1
removed 0.0 adsorbates! original coverage 1
sorting 3 adslabs, completed 2/4 iterations
grouped 1 sites to 1
symmetrically reduced to 1 adslabs from 1
removed 0.0 adsorbates! original coverage 1
sorting 4 adslabs, completed 3/4 iterations
coverage 1
partition_combos (0,) Total combos 1
groups 1 combinations into 1 groups
grouped 1 sites to 1
symmetrically reduced to 1 adslabs from 1
removed 0.0 adsorbates! original coverage 1
sorting 1 adslabs, completed 0/1 iterations
6
